In [1]:
import os
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import gmaps
import gmaps.geojson_geometries
import time
google_maps_api_key = os.environ.get('googlemaps_api_key')
gmaps.configure(api_key=google_maps_api_key)

In [ ]:
#import all data want mapped on google maps neighborhood overlay

In [99]:
df_value_transit = pd.read_csv('raw_data\\neighborhood_commute_and_transit_times.csv')
df_value_transit.drop('Unnamed: 0', axis='columns', inplace=True)
print(df_value_transit.shape)
df_value_transit.head(3)

(78, 40)


,Neighborhood,House Value,Median Rent,POPULATION_2010,PCT_WHITE,DENVER_REGION,drive-AM-to-Downtown,transit-AM-to-Downtown,drive-PM-from-Downtown,transit-PM-from-Downtown,...,% Slower Rush Hour Drive AM-to-DTC,Transit % Slower AM-to-DTC,% Slower Rush Hour Drive PM-from-DTC,Transit % Slower PM-from-DTC,% Slower Rush Hour Drive AM-to-CU-Boulder,Transit % Slower AM-to-CU-Boulder,% Slower Rush Hour Drive PM-from-CU-Boulder,Transit % Slower PM-from-CU-Boulder,Average % Slower Rush Hour Drive,Average Transit % Slower
0,Montbello,174000.0,1854.0,30348,8.17,Northeast,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,49.225092,124.727992,36.724566,220.931639,69.955379,100.408401,47.541766,106.599806,61.359118,124.838121
1,Gateway - Green Valley Ranch,208600.0,1951.0,29201,23.98,Northeast,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,56.443832,200.352423,35.878963,205.355249,16.809006,102.592223,4.315476,149.179743,48.168008,132.352911
2,Hampden,297900.0,2138.0,17547,66.96,Southeast,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,9.702660,107.560628,18.167457,108.288770,47.749900,106.280323,51.106026,59.111560,37.639157,99.242317


In [112]:
arrests_by_neighborhood = pd.read_csv('raw_data\\arrests_by_neighborhood.csv', 
                                     index_col=0)
print(arrests_by_neighborhood.shape)
arrests_by_neighborhood.head(3)

(79, 2)


,NEIGHBORHOOD_NAME,TOTAL_ARRESTS
0,Athmar Park,1353
1,Auraria,341
2,Baker,2163


In [113]:
citations_by_neighborhood = pd.read_csv('raw_data\\citations_by_neighborhood.csv', 
                                       index_col=0)
print(citations_by_neighborhood.shape)
citations_by_neighborhood.head(3)

(79, 2)


,NEIGHBORHOOD_NAME,TOTAL_CITATIONS
0,Athmar Park,2431
1,Auraria,579
2,Baker,4448


In [102]:
accidents_by_hood = pd.read_csv('raw_data\\accidents_by_hood.csv', 
                               index_col=0)
print(accidents_by_hood.shape)
accidents_by_hood.head(3)

(78, 6)


,NEIGHBORHOOD_ID,TOTAL_ACCIDENTS,NEIGHBORHOOD,DENVER_REGION,Unnamed: 2,Unnamed: 3
0,Athmar Park,1718,Athmar Park,West,NaN,NaN
1,Auraria,1284,Auraria,Northwest,NaN,NaN
2,Baker,7216,Baker,Central,NaN,NaN


In [103]:
bike_accs_df = pd.read_csv('raw_data\\bike_accs_df.csv', 
                          index_col=0)
print(bike_accs_df.shape)
bike_accs_df.head(3)

(79, 3)


,NEIGHBORHOOD_ID,BICYCLE_IND,AVG_ACCS_PER_YEAR
25,Five Points,116,23.2
9,Capitol Hill,100,20.0
8,CBD,86,17.2


In [104]:
ped_accs_df = pd.read_csv('raw_data\\ped_accs_df.csv', 
                         index_col=0)
print(ped_accs_df.shape)
ped_accs_df.head(3)

(79, 3)


,NEIGHBORHOOD_ID,PEDESTRIAN_IND,AVG_ACCS_PER_YEAR
9,Capitol Hill,193,38.6
63,Union Station,143,28.6
25,Five Points,142,28.4


In [66]:
#get Denver Neighborhood Geocode JSON to draw on gmaps

#Denver.gov has a shapefile of neighborhoods - convert it to GeocodeJSON so gmaps
#can import it by using mapshaper.org
#then import the Geocode JSON

with open('raw_data\\denver_neighborhood_outlines.json') as f:
    denver_neighborhood_geometry = json.load(f)

In [105]:
def gmaps_Denver_neighborhood_graph(neighb_series, plot_values_series, plot_value_title, shade_color):
    #normalize the plot_values to a list between 0.05 and 0.95 for opacity (minimum 
    #at 0.05 so at least some shading and maximum at 0.95 so almost full opacity, but can still see some features below)
    plot_values_max = plot_values_series.max()
    plot_values_min = plot_values_series.min()
    plot_values_range = plot_values_max - plot_values_min
    normalized_plot_values = plot_values_series.map(lambda x: 
                                0.05 + (x-plot_values_min)/plot_values_range*0.9)
    
    #create a dictionary of Neighborhood: Plot Values from df_series and normalized list
    neighborhood_plot_values_dict = dict(zip(neighb_series, normalized_plot_values))
    
    #cycle through the geojson in its order to create the list of opacity values
    opacity_plot_value = []
    for feature in denver_neighborhood_geometry.get('features'):
        neighborhood = feature.get('properties').get('NBHD_NAME')
        opacity_plot_value.append(neighborhood_plot_values_dict.get(neighborhood, 0)) #put in zero in case neighborhood data not there

    #overlay the Denver Neighborhoods on gmaps and use opacity as defined
    fig = gmaps.figure()

    denver_neighborhood_layer = gmaps.geojson_layer(
                                    denver_neighborhood_geometry,
                                    fill_color=shade_color,
                                    fill_opacity=opacity_plot_value
                                    )

    fig.add_layer(denver_neighborhood_layer)
    print(f"Maximum Opacity = {plot_values_max} {plot_value_title}")
    print(f"Minimum Opacity = {plot_values_min} {plot_value_title}")
    return fig

In [68]:
#Gmaps by Neighborhood of Median House Value
gmaps_Denver_neighborhood_graph(df_value_transit['Neighborhood'], 
                                df_value_transit['House Value'],
                                'Median House Value',
                                'purple')

Maximum Opacity = 898900.0 Median House Value
Minimum Opacity = 118300.0 Median House Value


Figure(layout=FigureLayout(height='420px'))

In [130]:
 df_value_transit.sort_values('House Value', ascending=False).head()

,Neighborhood,House Value,Median Rent,POPULATION_2010,PCT_WHITE,DENVER_REGION,drive-AM-to-Downtown,transit-AM-to-Downtown,drive-PM-from-Downtown,transit-PM-from-Downtown,...,% Slower Rush Hour Drive AM-to-DTC,Transit % Slower AM-to-DTC,% Slower Rush Hour Drive PM-from-DTC,Transit % Slower PM-from-DTC,% Slower Rush Hour Drive AM-to-CU-Boulder,Transit % Slower AM-to-CU-Boulder,% Slower Rush Hour Drive PM-from-CU-Boulder,Transit % Slower PM-from-CU-Boulder,Average % Slower Rush Hour Drive,Average Transit % Slower
68,Country Club,898900.0,4337.0,3001,91.30,Central,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,57.013118,92.866324,53.868472,174.481458,40.286110,58.388158,42.908438,56.595477,39.986430,100.687641
31,Hilltop,788600.0,3958.0,8190,84.19,East,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,21.439750,125.837629,39.170124,101.967800,35.941912,63.026706,41.213064,71.833700,29.860129,112.276079
53,Belcaro,768200.0,3259.0,4172,90.51,East,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,58.011696,128.793486,58.139535,148.088235,40.429420,78.399480,53.698162,72.016690,41.433873,117.316309
47,Cherry Creek,686500.0,3045.0,5589,87.60,Central,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,38.594970,147.747184,47.959184,141.896552,39.141414,62.613430,38.952071,62.320959,34.116108,102.061940
37,Washington Park,626300.0,2744.0,6905,92.05,South,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,60.467980,140.061397,47.678795,151.571793,42.972078,104.369414,40.704414,82.065906,33.969888,132.859106


In [69]:
#Gmaps by Neighborhood of Transit Slowing
gmaps_Denver_neighborhood_graph(df_value_transit['Neighborhood'], 
                                df_value_transit['Average Transit % Slower'],
                                '% Slower Commute is using Transit than Driving',
                                'olive')

Maximum Opacity = 201.5928712378069 % Slower Commute is using Transit than Driving
Minimum Opacity = 48.22667938675289 % Slower Commute is using Transit than Driving


Figure(layout=FigureLayout(height='420px'))

In [132]:
 df_value_transit.sort_values('Average Transit % Slower', ascending=False).head()

,Neighborhood,House Value,Median Rent,POPULATION_2010,PCT_WHITE,DENVER_REGION,drive-AM-to-Downtown,transit-AM-to-Downtown,drive-PM-from-Downtown,transit-PM-from-Downtown,...,% Slower Rush Hour Drive AM-to-DTC,Transit % Slower AM-to-DTC,% Slower Rush Hour Drive PM-from-DTC,Transit % Slower PM-from-DTC,% Slower Rush Hour Drive AM-to-CU-Boulder,Transit % Slower AM-to-CU-Boulder,% Slower Rush Hour Drive PM-from-CU-Boulder,Transit % Slower PM-from-CU-Boulder,Average % Slower Rush Hour Drive,Average Transit % Slower
16,Marston,220900.0,1939.0,11132,73.67,Southwest,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,26.292835,248.149975,26.682838,247.103877,14.480776,178.060264,25.849602,139.241597,28.929130,201.592871
61,Cory - Merrill,449400.0,2448.0,3892,88.39,East,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,46.675359,150.044444,48.015365,215.051903,41.423948,111.441648,51.305888,98.478642,35.526440,168.256548
30,Fort Logan,271700.0,2119.0,8532,63.44,Southwest,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,27.698309,199.083503,28.169014,249.650350,13.544259,149.436443,24.830054,119.231872,30.687217,168.168181
11,Speer,337200.0,1999.0,10954,81.90,Central,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,69.215292,118.846611,47.153700,157.640232,41.614907,67.780027,41.173641,68.858603,38.091179,154.119057
34,University Park,520800.0,2528.0,7491,83.47,South,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,29.277108,139.049394,52.924394,163.899254,42.926123,116.554054,56.229885,65.303119,40.969878,150.723993


In [106]:
#Gmaps by Neighborhood of # Accidents
gmaps_Denver_neighborhood_graph(accidents_by_hood['NEIGHBORHOOD_ID'], 
                                accidents_by_hood['TOTAL_ACCIDENTS'],
                                'Total # Traffic Accidents 2012 to Mid-2018',
                                'blue')

Maximum Opacity = 7984 Total # Traffic Accidents 2012 to Mid-2018
Minimum Opacity = 137 Total # Traffic Accidents 2012 to Mid-2018


Figure(layout=FigureLayout(height='420px'))

In [134]:
 accidents_by_hood.sort_values('TOTAL_ACCIDENTS', ascending=False).head()

,NEIGHBORHOOD_ID,TOTAL_ACCIDENTS,NEIGHBORHOOD,DENVER_REGION,Unnamed: 2,Unnamed: 3
59,Stapleton,7984,Stapleton,Northeast,NaN,NaN
2,Baker,7216,Baker,Central,NaN,NaN
25,Five Points,4797,Five Points,North,NaN,NaN
32,Hampden South,4712,Hampden South,Southeast,NaN,NaN
9,Capitol Hill,4480,Capitol Hill,Central,NaN,NaN


In [107]:
#Gmaps by Neighborhood of # Accidents Including Bikers
gmaps_Denver_neighborhood_graph(bike_accs_df['NEIGHBORHOOD_ID'], 
                                bike_accs_df['BICYCLE_IND'],
                                'Total # Accidents Including Bikers 2012 to Mid-2018',
                                'teal')

Maximum Opacity = 116 Total # Accidents Including Bikers 2012 to Mid-2018
Minimum Opacity = 0 Total # Accidents Including Bikers 2012 to Mid-2018


Figure(layout=FigureLayout(height='420px'))

In [136]:
 bike_accs_df.sort_values('BICYCLE_IND', ascending=False).head()

,NEIGHBORHOOD_ID,BICYCLE_IND,AVG_ACCS_PER_YEAR
25,Five Points,116,23.2
9,Capitol Hill,100,20.0
8,CBD,86,17.2
47,North Capitol Hill,82,16.4
63,Union Station,79,15.8


In [109]:
#Gmaps by Neighborhood of # Accidents Including Pedestrians
gmaps_Denver_neighborhood_graph(ped_accs_df['NEIGHBORHOOD_ID'], 
                                ped_accs_df['PEDESTRIAN_IND'],
                                'Total # Accidents Including Pedestrians 2012 to Mid-2018',
                                'aqua')

Maximum Opacity = 193 Total # Accidents Including Pedestrians 2012 to Mid-2018
Minimum Opacity = 0 Total # Accidents Including Pedestrians 2012 to Mid-2018


Figure(layout=FigureLayout(height='420px'))

In [138]:
ped_accs_df.sort_values('PEDESTRIAN_IND', ascending=False).head()

,NEIGHBORHOOD_ID,PEDESTRIAN_IND,AVG_ACCS_PER_YEAR
9,Capitol Hill,193,38.6
63,Union Station,143,28.6
25,Five Points,142,28.4
8,CBD,116,23.2
15,Civic Center,96,19.2


In [114]:
#Gmaps by Neighborhood of # Citations
gmaps_Denver_neighborhood_graph(citations_by_neighborhood['NEIGHBORHOOD_NAME'], 
                                citations_by_neighborhood['TOTAL_CITATIONS'],
                                'Total # Citations 2012 to Mid-2018',
                                'red')

Maximum Opacity = 4919 Total # Citations 2012 to Mid-2018
Minimum Opacity = 80 Total # Citations 2012 to Mid-2018


Figure(layout=FigureLayout(height='420px'))

In [140]:
citations_by_neighborhood.sort_values('TOTAL_CITATIONS', ascending=False).head()

,NEIGHBORHOOD_NAME,TOTAL_CITATIONS
44,Montbello,4919
60,Stapleton,4840
2,Baker,4448
3,Barnum,4203
76,Westwood,3965


In [115]:
#Gmaps by Neighborhood of # Arrests
gmaps_Denver_neighborhood_graph(arrests_by_neighborhood['NEIGHBORHOOD_NAME'], 
                                arrests_by_neighborhood['TOTAL_ARRESTS'],
                                'Total # Arrests 2012 to Mid-2018',
                                'green')

Maximum Opacity = 6671 Total # Arrests 2012 to Mid-2018
Minimum Opacity = 21 Total # Arrests 2012 to Mid-2018


Figure(layout=FigureLayout(height='420px'))

In [142]:
arrests_by_neighborhood.sort_values('TOTAL_ARRESTS', ascending=False).head()

,NEIGHBORHOOD_NAME,TOTAL_ARRESTS
23,East Colfax,6671
25,Five Points,6028
44,Montbello,3474
76,Westwood,3459
74,West Colfax,2754


In [44]:
###Additional Iterations Not Used Below###

In [8]:
# ###initial iteration without function###
# #make the opacity of the neighborhood fill higher for higher price neighborhoods

# #normalize the House Value to a list between 0.1 and 1.0 for opacity (minimum 
# #at 0.1 so at least some shading and maximum at 1.0 so full opacity)
# house_max = df['House Value'].max()
# house_min = df['House Value'].min()
# house_range = house_max - house_min
# normalized_house_value = df['House Value'].map(lambda x: 
#                                 0.1 + (x-house_min)/house_range*0.9)

# #create a dictionary of Neighborhood: House Value from df  and normalized list
# neighborhood_house_price_dict = dict(zip(df['Neighborhood'], 
#                                          normalized_house_value))

# #cycle through the geojson in its order to create the list of opacity values
# opacity_house_value = []
# for feature in denver_neighborhood_geometry.get('features'):
#     neighborhood = feature.get('properties').get('NBHD_NAME')
#     opacity_house_value.append(neighborhood_house_price_dict.get(neighborhood))

# #overlay the Denver Neighborhoods on gmaps and use opacity as defined
# fig = gmaps.figure()

# denver_neighborhood_layer = gmaps.geojson_layer(
#                                 denver_neighborhood_geometry,
#                                 fill_color='red',
#                                 fill_opacity=opacity_house_value
#                                 )

# fig.add_layer(denver_neighborhood_layer)
# fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# #get lat/long of each neighborhood center so can place markers with information

# #https://maps.googleapis.com/maps/api/geocode/outputFormat?parameters
# def get_neighborhood_center_lat_long(neighborhood_name):
#     base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
#     parameters = {'address': f"{neighborhood_name} Neighborhood, Denver, CO", 
#                   'key': google_maps_api_key
#                   }

#     response = requests.get(base_url, parameters)
#     response_json = response.json()
#     return response_json

In [38]:
# df_markers = pd.DataFrame(df_value_transit['Neighborhood'])
# df_markers.head()

,Neighborhood
0,Montbello
1,Gateway - Green Valley Ranch
2,Hampden
3,Westwood
4,Capitol Hill


In [39]:
# #call response from Google Geocode API and parse out center lat/long of each neighborhood in df
# df_markers['Neighb Geo JSON']=df_markers['Neighborhood'].map(get_neighborhood_center_lat_long)
# df_markers['lat']=df_markers['Neighb Geo JSON'].map(lambda x:
#                     x.get('results')[0].get('geometry').get('location').get('lat'))
# df_markers['long']=df_markers['Neighb Geo JSON'].map(lambda x:
#                     x.get('results')[0].get('geometry').get('location').get('lng'))
# df_markers.head()

,Neighborhood,Neighb Geo JSON,lat,long
0,Montbello,{'results': [{'address_components': [{'long_na...,39.782702,-104.839625
1,Gateway - Green Valley Ranch,{'results': [{'address_components': [{'long_na...,39.785265,-104.799937
2,Hampden,{'results': [{'address_components': [{'long_na...,39.657695,-104.889247
3,Westwood,{'results': [{'address_components': [{'long_na...,39.703339,-105.041130
4,Capitol Hill,{'results': [{'address_components': [{'long_na...,39.734114,-104.979775


In [43]:
# ##markers make the maps look pretty ugly so don't actually use them to show neihborhood info##

# #overlay the Denver Neighborhoods on gmaps and add markers that show their names on click
# fig = gmaps.figure()

# denver_neighborhood_layer = gmaps.geojson_layer(
#                                 denver_neighborhood_geometry,
#                                 fill_color='gray',
#                                 fill_opacity=0.2
#                                 )

# fig.add_layer(denver_neighborhood_layer)


# markers_names = gmaps.marker_layer(
#                                 locations=list(zip(df_markers['lat'], df_markers['long'])), 
#                                 info_box_content=df_markers['Neighborhood']) #click will give Neighborhood Name

# fig.add_layer(markers_names)

# fig

Figure(layout=FigureLayout(height='420px'))